In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
%matplotlib inline

import os
import statistics
import nltk
import re

from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.charfilter import *
from janome.tokenfilter import *

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json

from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


# TEXT DATASET

TRAINING DATA

In [2]:
# TRAINING SET
#Bankrupt_risk
bankrupted_risk_reports = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/bankrupted/trainB')
bankrupted_risk_reports = [file for file in bankrupted_risk_reports]

bankrupted_risk_file = []
for report in range(0, len(bankrupted_risk_reports)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/bankrupted/trainB/{bankrupted_risk_reports[report]}", encoding="utf8") as file:
        risk_filesB = file.read()
        bankrupted_risk_file.append(risk_filesB)

#Bankrupt_tanshin
bankrupted_tanshin_reports = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/bankrupted/trainB')
bankrupted_tanshin_reports = [file for file in bankrupted_tanshin_reports]

bankrupted_file_tanshin = []
for report in range(0, len(bankrupted_tanshin_reports)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/bankrupted/trainB/{bankrupted_tanshin_reports[report]}", encoding="utf8") as file:
        filesB_tanshin = file.read()
        bankrupted_file_tanshin.append(filesB_tanshin)


df_bankrupted_risk = pd.DataFrame(bankrupted_risk_file)
df_bankrupted_risk.columns = ['risk']
df_bankrupted_risk['Y'] = 0 #Y=0 means bankrupted

df_bankrupted_tanshin = pd.DataFrame(bankrupted_file_tanshin)
df_bankrupted_tanshin.columns = ['tanshin']
df_bankrupted_tanshin['Y'] = 0 


#Nonbankrupt_risk
nonbankrupted_risk_reports = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/nonbankrupted/trainNB')
nonbankrupted_risk_reports = [file for file in nonbankrupted_risk_reports]

nonbankrupted_risk_file = []
for report in range(0, len(nonbankrupted_risk_reports)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/nonbankrupted/trainNB/{nonbankrupted_risk_reports[report]}", encoding="utf8") as file:
        risk_filesNB = file.read()
        nonbankrupted_risk_file.append(risk_filesNB)

#Nonbankrupt__tanshin
nonbankrupted_tanshin_reports = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/nonbankrupted/trainNB')
nonbankrupted_tanshin_reports = [file for file in nonbankrupted_tanshin_reports]

nonbankrupted_file_tanshin = []
for report in range(0, len(nonbankrupted_tanshin_reports)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/nonbankrupted/trainNB/{nonbankrupted_tanshin_reports[report]}", encoding="utf8") as file:
        filesNB_tanshin = file.read()
        nonbankrupted_file_tanshin.append(filesNB_tanshin) 
        
df_nonbankrupted_risk = pd.DataFrame(nonbankrupted_risk_file)
df_nonbankrupted_risk.columns = ['risk']
df_nonbankrupted_risk['Y'] = 1

df_nonbankrupted_tanshin = pd.DataFrame(nonbankrupted_file_tanshin)
df_nonbankrupted_tanshin.columns = ['tanshin']
df_nonbankrupted_tanshin['Y'] = 1


# concatenated_train
df_risk_train = [df_bankrupted_risk,df_nonbankrupted_risk]
df_risk_train = pd.concat(df_risk_train, ignore_index=True)
# print(df_risk_train)

concatenated_tanshin_train = [df_bankrupted_tanshin, df_nonbankrupted_tanshin]
df_tanshin_train = pd.concat(concatenated_tanshin_train, ignore_index= True)
# print(df_tanshin_train)

# Combine the texts of same company
df_train = df_risk_train
df_train['tanshin'] = df_tanshin_train['tanshin']
df_train['texts'] = df_train[['risk','tanshin']].apply(lambda x: '　'.join(x), axis=1)
df_train = df_train.drop('risk', axis=1)
df_train = df_train.drop('tanshin', axis=1)
# print(df_train)

VALIDATION DATA

In [3]:
# VALIDATION SET
#Bankrupt_risk
bankrupted_risk_reports_val = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/bankrupted/valB')
bankrupted_risk_reports_val = [file for file in bankrupted_risk_reports_val]

bankrupted_risk_file_val = []
for report in range(0, len(bankrupted_risk_reports_val)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/bankrupted/valB/{bankrupted_risk_reports_val[report]}", encoding="utf8") as file:
        risk_filesB_val = file.read()
        bankrupted_risk_file_val.append(risk_filesB_val)
        
#Bankrupt_tanshin
bankrupted_tanshin_reports_val = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/bankrupted/valB')
bankrupted_tanshin_reports_val = [file for file in bankrupted_tanshin_reports_val]

bankrupted_file_tanshin_val = []
for report in range(0, len(bankrupted_tanshin_reports_val)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/bankrupted/valB/{bankrupted_tanshin_reports_val[report]}", encoding="utf8") as file:
        filesB_tanshin_val = file.read()
        bankrupted_file_tanshin_val.append(filesB_tanshin_val)       
        
df_bankrupted_risk_val = pd.DataFrame(bankrupted_risk_file_val)
df_bankrupted_risk_val.columns = ['risk']
df_bankrupted_risk_val['Y'] = 0 #Y=0 means bankrupted

df_bankrupted_tanshin_val = pd.DataFrame(bankrupted_file_tanshin_val)
df_bankrupted_tanshin_val.columns = ['tanshin']
df_bankrupted_tanshin_val['Y'] = 0


#Nonbankrupt_risk
nonbankrupted_risk_reports_val = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/nonbankrupted/valNB')
nonbankrupted_risk_reports_val = [file for file in nonbankrupted_risk_reports_val]

nonbankrupted_risk_file_val = []
for report in range(0, len(nonbankrupted_risk_reports_val)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/nonbankrupted/valNB/{nonbankrupted_risk_reports_val[report]}", encoding="utf8") as file:
        risk_filesNB_val = file.read()
        nonbankrupted_risk_file_val.append(risk_filesNB_val)
        
#Nonbankrupt_tanshin
nonbankrupted_tanshin_reports_val = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/nonbankrupted/valNB')
nonbankrupted_tanshin_reports_val = [file for file in nonbankrupted_tanshin_reports_val]

nonbankrupted_file_tanshin_val = []
for report in range(0, len(nonbankrupted_tanshin_reports_val)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/nonbankrupted/valNB/{nonbankrupted_tanshin_reports_val[report]}", encoding="utf8") as file:
        filesNB_tanshin_val = file.read()
        nonbankrupted_file_tanshin_val.append(filesNB_tanshin_val)
        
df_nonbankrupted_risk_val = pd.DataFrame(nonbankrupted_risk_file_val)
df_nonbankrupted_risk_val.columns = ['risk']
df_nonbankrupted_risk_val['Y'] = 1

df_nonbankrupted_tanshin_val = pd.DataFrame(nonbankrupted_file_tanshin_val)
df_nonbankrupted_tanshin_val.columns = ['tanshin']
df_nonbankrupted_tanshin_val['Y'] = 1 


# concatenated_val
df_risk_val = [df_bankrupted_risk_val,df_nonbankrupted_risk_val]
df_risk_val = pd.concat(df_risk_val, ignore_index=True)
# print(df_risk_val)

concatenated_tanshin_val = [df_bankrupted_tanshin_val, df_nonbankrupted_tanshin_val]
df_tanshin_val = pd.concat(concatenated_tanshin_val, ignore_index=True)
# print(df_tanshin_val) 

# Combine the texts of same company
df_val = df_risk_val
df_val['tanshin'] = df_tanshin_val['tanshin']
df_val['texts'] = df_val[['risk','tanshin']].apply(lambda x: '　'.join(x), axis=1)
df_val = df_val.drop('risk', axis=1)
df_val = df_val.drop('tanshin', axis=1)
# df_val

TEST DATA

In [4]:
# TEST SET

#Bankrupt_risk (test data)
bankrupted_risk_reports_test = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/bankrupted/testB')

bankrupted_risk_file_test = []
for report in range(0, len(bankrupted_risk_reports_test)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/bankrupted/testB/{bankrupted_risk_reports_test[report]}", encoding="utf8") as file:
        filesB_test = file.read()
        bankrupted_risk_file_test.append(filesB_test)    

#Bankrupt_tanshin(test data)
bankrupted_tanshin_reports_test = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/bankrupted/testB')
bankrupted_tanshin_reports_test = [file for file in bankrupted_tanshin_reports_test]

bankrupted_tanshin_file_test = []
for report in range(0, len(bankrupted_tanshin_reports_test)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/bankrupted/testB/{bankrupted_tanshin_reports_test[report]}", encoding="utf8") as file:
        tanshin_filesB_test = file.read()
        bankrupted_tanshin_file_test.append(tanshin_filesB_test)
        
df_bankrupted_risk_test = pd.DataFrame(bankrupted_risk_file_test)
df_bankrupted_risk_test.columns = ['risk']
df_bankrupted_risk_test['Y'] = 0 #Y=0 means bankrupted

df_bankrupted_tanshin_test = pd.DataFrame(bankrupted_tanshin_file_test)
df_bankrupted_tanshin_test.columns = ['tanshin']
df_bankrupted_tanshin_test['Y'] = 0

#Nonbankrupt_risk (test data)
nonbankrupted_risk_reports_test = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/nonbankrupted/testNB')
nonbankrupted_risk_reports_test = [file for file in nonbankrupted_risk_reports_test]

nonbankrupted_risk_file_test = []
for report in range(0, len(nonbankrupted_risk_reports_test)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/risk_info/nonbankrupted/testNB/{nonbankrupted_risk_reports_test[report]}", encoding="utf8") as file:
        risk_filesNB_test = file.read()
        nonbankrupted_risk_file_test.append(risk_filesNB_test)    
        
#Nonbankrupt_tanshin(test data)
nonbankrupted_tanshin_reports_test = os.listdir('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/nonbankrupted/testNB')
nonbankrupted_tanshin_reports_test = [file for file in nonbankrupted_tanshin_reports_test]

nonbankrupted_tanshin_file_test = []
for report in range(0, len(nonbankrupted_tanshin_reports_test)):
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/tanshin/nonbankrupted/testNB/{nonbankrupted_tanshin_reports_test[report]}", encoding="utf8") as file:
        tanshin_filesNB_test = file.read()
        nonbankrupted_tanshin_file_test.append(tanshin_filesNB_test)

df_nonbankrupted_risk_test = pd.DataFrame(nonbankrupted_risk_file_test)
df_nonbankrupted_risk_test.columns = ['risk']
df_nonbankrupted_risk_test['Y'] = 1

df_nonbankrupted_tanshin_test = pd.DataFrame(nonbankrupted_tanshin_file_test)
df_nonbankrupted_tanshin_test.columns = ['tanshin']
df_nonbankrupted_tanshin_test['Y'] = 1

# concatenated_test
df_risk_test = [df_bankrupted_risk_test,df_nonbankrupted_risk_test]
df_risk_test = pd.concat(df_risk_test, ignore_index=True)
# print(df_risk_test)

df_tanshin_test = [df_bankrupted_tanshin_test,df_nonbankrupted_tanshin_test]
df_tanshin_test = pd.concat(df_tanshin_test, ignore_index=True)
# print(df_tanshin_test)

# Combine the texts of same company
df_test = df_risk_test
df_test['tanshin'] = df_tanshin_test['tanshin']
df_test['texts'] = df_test[['risk','tanshin']].apply(lambda x: '　'.join(x), axis=1)
df_test = df_test.drop('risk', axis=1)
df_test = df_test.drop('tanshin', axis=1)
# df_test

Concatenated_files

In [5]:
df = [df_train, df_val, df_test]
df = pd.concat(df, ignore_index=True)
# df

In [6]:
df['length'] = df['texts'].apply(len)
# df.head()

# TEXT PREPROCESSING

In [7]:
def clean_data(risk_report_test):
    """Returns cleaned text data."""
    
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/stop_words.txt", mode="r", encoding="utf8") as file:
        stop_words = file.read()
    
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/stop_words_accounting.txt", mode="r", encoding="utf8") as file:
        stop_words_acc = file.read()
    
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/onaji_risk_vocab.txt", mode="r", encoding="utf8") as file:
        onaji_risk_vocab = file.read()
    
    with open(f"C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/onaji_tanshin_vocab.txt", mode="r", encoding="utf8") as file:
        onaji_tanshin_vocab = file.read()  
    
    char_filters = [UnicodeNormalizeCharFilter(), 
                    RegexReplaceCharFilter(u'キャッシュ・フロー', u'キャッシュフロー')]
    tokenizer = Tokenizer()
    token_filters = [CompoundNounFilter(), 
                     POSStopFilter(['記号','助詞', '数', '数接続', '接続詞','連体詞', '接頭詞', '名詞,数','非自立', '代名詞', '自動詞', '他動詞']), 
                     LowerCaseFilter()]


    a = Analyzer(char_filters=char_filters, tokenizer=tokenizer, token_filters=token_filters)
    
    def filter(text):
        """
        :param text: str
        :rtype : str
        """
       # アルファベットと半角英数と記号と改行とタブを排除
        text = re.sub(r'[a-zA-Z0-9¥"¥.¥,¥@]+', '', text)
        text = re.sub(r'[!"“#$%&()\*\+\-\.,\/:;<=>?@\[\\\]^_`{|}~]', '', text)
        text = re.sub(r'[\n|\r|\t]', '', text)
        
        # 日本語以外の文字を排除
        jp_chartype_tokenizer = nltk.RegexpTokenizer(u'([ぁ-んー]+|[ァ-ンー]+|[\u4e00-\u9FFF]+|[ぁ-んァ-ンー\u4e00-\u9FFF]+)')
        text = ''.join(jp_chartype_tokenizer.tokenize(text))
        
        return text

    nosymbol_test = filter(risk_report_test)
    
    doc = []
    for token in a.analyze(nosymbol_test):
        doc.append(token.surface)
    doc = [x for x in doc if x not in stop_words]
    doc = [x for x in doc if x not in stop_words_acc]
    doc = [x for x in doc if x not in onaji_risk_vocab]
    doc = [x for x in doc if x not in onaji_tanshin_vocab]
    doc = np.array(doc)
    
    return doc

BAG OF WORDS AND TF-IDF

In [8]:
clean_file_transformer = CountVectorizer(analyzer=clean_data).fit(df['texts'])
# print(clean_file_transformer.get_feature_names())

In [9]:
clean_file_bow = clean_file_transformer.transform(df['texts'])
# print(clean_file_bow)
print('Lenght of clean file transformer vocab:', len(clean_file_transformer.vocabulary_))
print('Shape of Sparse Matrix: ', clean_file_bow.shape)

Lenght of clean file transformer vocab: 14045
Shape of Sparse Matrix:  (137, 14045)


In [10]:
transformer_tfidf = TfidfTransformer().fit(clean_file_bow)
clean_file_tfidf = transformer_tfidf.transform(clean_file_bow)

# print(clean_file_tfidf)
print(clean_file_tfidf.shape)

(137, 14045)


# LOAD TEXT CLASSIFIER

TRAIN/VAL/TEST DATASETS

In [11]:
X = clean_file_tfidf
X = pd.DataFrame.sparse.from_spmatrix(X)
X_train = X[:92]
X_val = X[92:112]
X_test = X[112:]

y = np.array(df['Y'])
y_train = y[:92]
y_val = y[92:112]
y_test = y[112:]

print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)

(92, 14045) (20, 14045) (25, 14045) (92,) (20,) (25,)


CLASSIFIER (Neural network)

In [12]:
# load json and create model
json_file = open('modelD.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model_D_ens_T80.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test, y_test, verbose=0)

print('TEXT:', "%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
TEXT: accuracy: 80.00%


EVALUATE MODELS ON TEST DATA

In [13]:
predict_text = []

for predictions in loaded_model.predict(X_test):
    if predictions < 0.5:
        predictions = 0
    else:
        predictions = 1
    predict_text.append(predictions)
    
pred = np.array(predict_text)
true = np.array(y_test)

# print(classification_report(true, pred))
print('\n CONFUSION MATRIX:\n', confusion_matrix(true, pred))
print('\n\n ACCURACY SCORE IS:', accuracy_score(true, pred))


 CONFUSION MATRIX:
 [[12  0]
 [ 5  8]]


 ACCURACY SCORE IS: 0.8


# RATIO DATASET

In [14]:
# TRAINING SET

traindata = pd.read_csv('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/ratios/train_ratio.csv')
# print('TRAINING DATASET \n:', traindata.describe().transpose())
X_train_ratio = traindata.drop('Y', axis=1)
y_train_ratio = traindata['Y']
# print(sns.countplot(x = 'Y', data=traindata))

# VALIDATION SET

valdata = pd.read_csv('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/ratios/val_ratio.csv')
# print('VALIDATION DATASET \n:', valdata.describe().transpose())
X_val_ratio = valdata.drop('Y', axis=1)
y_val_ratio = valdata['Y']

# TEST SET

testdata = pd.read_csv('C:/Users/onjad/OneDrive/Documents/RESEARCH PHD/article 3/CODES/DATAFRAMES/ratios/test_ratio.csv')
# print('TEST DATASET \n:', testdata.describe().transpose())
# print('TEST DATASET \n:', testdata)
X_test_ratio = testdata.drop('Y', axis=1)
y_test_ratio = testdata['Y']

print('Length of X (train): {} | Length of Y (train): {}'.format(len(X_train_ratio), len(y_train_ratio)))
print('Length of X (validation): {} | Length of Y (validation): {}'.format(len(X_val_ratio), len(y_val_ratio)))
print('Length of X (test): {} | Length of Y (test): {}'.format(len(X_test_ratio), len(y_test_ratio)))

Length of X (train): 92 | Length of Y (train): 92
Length of X (validation): 20 | Length of Y (validation): 20
Length of X (test): 25 | Length of Y (test): 25


# RATIO DATA PREPROCESSING

In [15]:
scaler = MinMaxScaler()
X_train_ratio = scaler.fit_transform(X_train_ratio)
X_val_ratio = scaler.transform(X_val_ratio)
X_test_ratio = scaler.transform(X_test_ratio)

In [16]:
Xtrain_ratio = pd.DataFrame(X_train_ratio)
Xval_ratio = pd.DataFrame(X_val_ratio)
Xtest_ratio = pd.DataFrame(X_test_ratio)

# rename columns
Xtrain_ratio.columns = ['X19','X20','X29','X47', 'X51', 'X54', 'X59', 'X62','X63','X64','X67', 'X69', 'X70', 'X71','X72', 'X73', 'X74', 'X76','X77', 'X79', 'X80', 'X84']
Xval_ratio.columns = ['X19','X20','X29','X47', 'X51', 'X54', 'X59', 'X62','X63','X64','X67', 'X69', 'X70', 'X71','X72', 'X73', 'X74', 'X76','X77', 'X79', 'X80', 'X84']
Xtest_ratio.columns = ['X19','X20','X29','X47', 'X51', 'X54', 'X59', 'X62','X63','X64','X67', 'X69', 'X70', 'X71','X72', 'X73', 'X74', 'X76','X77', 'X79', 'X80', 'X84']

# TARGET VALUE Y

y_train_ratio = pd.DataFrame(y_train_ratio)
y_val_ratio = pd.DataFrame(y_val_ratio)
y_test_ratio = pd.DataFrame(y_test_ratio)

print('x ratio:', Xtrain_ratio.shape, Xval_ratio.shape, Xtest_ratio.shape)
print('y ratio:', y_train_ratio.shape, y_val_ratio.shape, y_test_ratio.shape)

x ratio: (92, 22) (20, 22) (25, 22)
y ratio: (92, 1) (20, 1) (25, 1)


# LOAD RATIO CLASSIFIER

In [17]:
# load json and create model
json_fileR = open('modelR.json', 'r')
loaded_modelR_json = json_fileR.read()
json_fileR.close()
loaded_modelR = model_from_json(loaded_modelR_json)

# load weights into new model
loaded_modelR.load_weights("model_ratio72.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_modelR.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
scoreR = loaded_modelR.evaluate(Xtest_ratio, y_test_ratio, verbose=0)

print('RATIO:', "%s: %.2f%%" % (loaded_modelR.metrics_names[1], scoreR[1]*100))

Loaded model from disk
RATIO: accuracy: 72.00%


In [18]:
predict_ratio = []

for predictions in loaded_modelR.predict(Xtest_ratio):
    if predictions < 0.5:
        predictions = 0
    else:
        predictions = 1
    predict_ratio.append(predictions)
    
predr = np.array(predict_ratio)
truer = np.array(y_test_ratio)

print('\n CONFUSION MATRIX:\n', confusion_matrix(truer, predr))
print('\n\n ACCURACY SCORE IS:', accuracy_score(truer, predr))


 CONFUSION MATRIX:
 [[10  2]
 [ 5  8]]


 ACCURACY SCORE IS: 0.72


# ENSEMBLE MODEL (SAME WEIGHT)

In [19]:
# Average probabilistic predictions of both models

pred_NN_ratio = []
pred_NN_text = []

for predictions in loaded_modelR.predict(Xtest_ratio):
    pred_NN_ratio.append(predictions)

for predictionss in loaded_model.predict(X_test):
    pred_NN_text.append(predictionss)
    
pred_ratio = np.array(pred_NN_ratio)
pred_ratio = pred_ratio.flatten()
pred_text = np.array(pred_NN_text)
pred_text = pred_text.flatten()
    
print('ratio pred:', pred_ratio)
print('text pred:', pred_text)
print('true pred:', np.array(y_test))

average = [statistics.mean(k) for k in zip(pred_ratio, pred_text)]
print('average:', average)

ratio pred: [0.29413337 0.32434452 0.46695027 0.41021785 0.62417907 0.6484067
 0.44789675 0.04360136 0.47557557 0.34017232 0.04245704 0.4046558
 0.7007073  0.00151616 0.19504988 0.562726   0.24516591 0.9659474
 0.19825754 0.7409878  0.575717   0.47162014 0.6109704  0.74966276
 0.8839359 ]
text pred: [0.4435838  0.45850664 0.15574753 0.42444578 0.4140469  0.39037588
 0.1308079  0.10074174 0.49064645 0.24814048 0.26721546 0.3494853
 0.8515818  0.50502145 0.8528352  0.07900676 0.37095484 0.63143325
 0.276604   0.3556914  0.10398054 0.8944074  0.878916   0.69266117
 0.85533184]
true pred: [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
average: [0.36885858, 0.39142558, 0.31134892, 0.4173318, 0.519113, 0.5193913, 0.28935233, 0.072171554, 0.48311102, 0.2941564, 0.15483625, 0.37707055, 0.77614456, 0.2532688, 0.52394253, 0.3208664, 0.30806038, 0.7986903, 0.23743077, 0.5483396, 0.33984876, 0.6830138, 0.7449432, 0.72116196, 0.8696339]


In [20]:
# PERFORMANCE EVALUATION - AVERAGE

pred_overall = []

for predictions in average:
    if predictions < 0.50:
        predictions = 0
    else:
        predictions=1
    pred_overall.append(predictions)
    
predict = np.array(pred_overall)
actual = np.array(y_test)

print(classification_report(actual, predict))
print('\n CONFUSION MATRIX:\n', confusion_matrix(actual, predict))
print('\n\n ACCURACY SCORE IS:', accuracy_score(actual, predict))

              precision    recall  f1-score   support

           0       0.67      0.83      0.74        12
           1       0.80      0.62      0.70        13

    accuracy                           0.72        25
   macro avg       0.73      0.72      0.72        25
weighted avg       0.74      0.72      0.72        25


 CONFUSION MATRIX:
 [[10  2]
 [ 5  8]]


 ACCURACY SCORE IS: 0.72


# ENSEMBLE MODEL (DIFFERENT WEIGHTS)

In [21]:
# Grid search best weight combination

pred_NN_ratio_weighted = []
pred_NN_text_weighted = []

for predictions in loaded_modelR.predict(Xtest_ratio):
    pred_NN_ratio_weighted.append(predictions)

for predictionss in loaded_model.predict(X_test):
    pred_NN_text_weighted.append(predictionss)
    
pred_ratio_weighted = np.array(pred_NN_ratio_weighted)
pred_ratio_weighted = pred_ratio_weighted.flatten()
pred_text_weighted = np.array(pred_NN_text_weighted)
pred_text_weighted = pred_text_weighted.flatten()

for w_ratio in np.arange(0, 1, 0.0001):
    w_text = 1 - w_ratio
    weighted_average = [np.sum(k) for k in zip((pred_ratio_weighted*w_ratio), (pred_text_weighted*w_text))]
#     print('\n Weighted average with distribution',w_ratio,'/', w_text, 'is:', weighted_average)
    pred_weighted = []
    for predictions in weighted_average:
        if predictions < 0.50:
            predictions = 0
        else:
            predictions = 1
        pred_weighted.append(predictions)
    pred_weighted = np.array(pred_weighted)
    actual = np.array(y_test) 
    print('ACCURACY ENSEMBLE WITH distribution', w_ratio,'/', w_text, 'is:', accuracy_score(actual, pred_weighted))                                  

ACCURACY ENSEMBLE WITH distribution 0.0 / 1.0 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0001 / 0.9999 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0002 / 0.9998 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.00030000000000000003 / 0.9997 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0004 / 0.9996 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0005 / 0.9995 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0006000000000000001 / 0.9994 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0007 / 0.9993 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0008 / 0.9992 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0009000000000000001 / 0.9991 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.001 / 0.999 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0011 / 0.9989 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0012000000000000001 / 0.9988 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0013000000000000002 / 0.9987 is: 0.8
ACCURACY ENSEMBLE WITH distribution 0.0014 / 0.9986 is: 0.8
ACCURACY ENSEMBLE WITH distribut

ACCURACY ENSEMBLE WITH distribution 0.0177 / 0.9823 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0178 / 0.9822 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0179 / 0.9821 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.018000000000000002 / 0.982 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0181 / 0.9819 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0182 / 0.9818 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0183 / 0.9817 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0184 / 0.9816 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.018500000000000003 / 0.9815 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.018600000000000002 / 0.9814 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0187 / 0.9813 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0188 / 0.9812 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0189 / 0.9811 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.019 / 0.981 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.019100000000000002 / 0.9809 is: 0.76
ACCURACY ENSEMBLE WITH distribut

ACCURACY ENSEMBLE WITH distribution 0.0356 / 0.9644 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0357 / 0.9643 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0358 / 0.9642 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0359 / 0.9641 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.036000000000000004 / 0.964 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0361 / 0.9639 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0362 / 0.9638 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0363 / 0.9637 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0364 / 0.9636 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.036500000000000005 / 0.9635 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0366 / 0.9634 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0367 / 0.9633 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0368 / 0.9632000000000001 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0369 / 0.9631 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.037000000000000005 / 0.963 is: 0.76
ACCURACY ENSEMBLE WITH distributi

ACCURACY ENSEMBLE WITH distribution 0.0529 / 0.9471 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.053000000000000005 / 0.947 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0531 / 0.9469 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.053200000000000004 / 0.9468 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0533 / 0.9467 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0534 / 0.9466 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.053500000000000006 / 0.9465 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0536 / 0.9464 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.053700000000000005 / 0.9463 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0538 / 0.9462 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0539 / 0.9460999999999999 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.054 / 0.946 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0541 / 0.9459 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.054200000000000005 / 0.9458 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0543 / 0.9457 is: 0.76
ACCURA

ACCURACY ENSEMBLE WITH distribution 0.0716 / 0.9284 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0717 / 0.9283 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0718 / 0.9282 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0719 / 0.9281 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.07200000000000001 / 0.9279999999999999 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0721 / 0.9279 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0722 / 0.9278 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0723 / 0.9277 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0724 / 0.9276 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.07250000000000001 / 0.9275 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0726 / 0.9274 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0727 / 0.9273 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0728 / 0.9272 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.0729 / 0.9271 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.07300000000000001 / 0.927 is: 0.76
ACCURACY ENSEMBLE WITH distribution

...

ACCURACY ENSEMBLE WITH distribution 0.9154 / 0.08460000000000001 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9155000000000001 / 0.08449999999999991 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9156000000000001 / 0.08439999999999992 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9157000000000001 / 0.08429999999999993 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9158000000000001 / 0.08419999999999994 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9159 / 0.08409999999999995 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.916 / 0.08399999999999996 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9161 / 0.08389999999999997 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9162 / 0.08379999999999999 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9163 / 0.0837 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9164 / 0.08360000000000001 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9165000000000001 / 0.08349999999999991 is: 0.76
ACCURACY ENSEMBLE WITH distribution 0.9166000000000001 / 0.0833999

ACCURACY ENSEMBLE WITH distribution 0.9354 / 0.06459999999999999 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9355 / 0.0645 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9356000000000001 / 0.0643999999999999 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9357000000000001 / 0.06429999999999991 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9358000000000001 / 0.06419999999999992 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9359000000000001 / 0.06409999999999993 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.936 / 0.06399999999999995 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9361 / 0.06389999999999996 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9362 / 0.06379999999999997 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9363 / 0.06369999999999998 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9364 / 0.06359999999999999 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9365 / 0.0635 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9366000000000001 / 0.0633999999999999 is: 0.72
ACCURAC

ACCURACY ENSEMBLE WITH distribution 0.9532 / 0.04679999999999995 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9533 / 0.046699999999999964 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9534 / 0.046599999999999975 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9535 / 0.046499999999999986 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9536 / 0.0464 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9537 / 0.04630000000000001 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9538000000000001 / 0.04619999999999991 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9539000000000001 / 0.04609999999999992 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9540000000000001 / 0.04599999999999993 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9541000000000001 / 0.04589999999999994 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9542 / 0.04579999999999995 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9543 / 0.04569999999999996 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9544 / 0.045599999999999974 is: 0.72

ACCURACY ENSEMBLE WITH distribution 0.9717 / 0.028299999999999992 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9718 / 0.028200000000000003 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9719000000000001 / 0.028099999999999903 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9720000000000001 / 0.027999999999999914 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9721000000000001 / 0.027899999999999925 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9722000000000001 / 0.027799999999999936 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9723 / 0.027699999999999947 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9724 / 0.027599999999999958 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9725 / 0.02749999999999997 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9726 / 0.02739999999999998 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9727 / 0.02729999999999999 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9728 / 0.027200000000000002 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9729000000000001 

ACCURACY ENSEMBLE WITH distribution 0.9904000000000001 / 0.009599999999999942 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9905 / 0.009499999999999953 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9906 / 0.009399999999999964 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9907 / 0.009299999999999975 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9908 / 0.009199999999999986 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9909 / 0.009099999999999997 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.991 / 0.009000000000000008 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9911000000000001 / 0.008899999999999908 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9912000000000001 / 0.008799999999999919 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9913000000000001 / 0.00869999999999993 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9914000000000001 / 0.008599999999999941 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9915 / 0.008499999999999952 is: 0.72
ACCURACY ENSEMBLE WITH distribution 0.9916

In [22]:
best_weight_ratio = 0.3746
best_weight_text = 0.6254

best_weighted_average = [np.sum(k) for k in zip((pred_ratio_weighted*best_weight_ratio), (pred_text_weighted*best_weight_text))]

pred_best_weight = []

for predictions in best_weighted_average:
    if predictions < 0.50:
        predictions = 0
    else:
        predictions=1
    pred_best_weight.append(predictions)
    
pred_best_weight = np.array(pred_best_weight)

print('Best weight distribution is:', best_weight_ratio,'/', best_weight_text,
      'with an accuracy score of', accuracy_score(actual, pred_best_weight))
print(classification_report(actual, pred_best_weight))
print('\n CONFUSION MATRIX:\n', confusion_matrix(actual, pred_best_weight))

Best weight distribution is: 0.3746 / 0.6254 with an accuracy score of 0.8
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        12
           1       1.00      0.62      0.76        13

    accuracy                           0.80        25
   macro avg       0.85      0.81      0.79        25
weighted avg       0.86      0.80      0.79        25


 CONFUSION MATRIX:
 [[12  0]
 [ 5  8]]


# COMPARISON OF ALL MODELS

In [23]:
print('RATIO ONLY ACCURACY SCORE IS: %0.2f' % (100 * accuracy_score(truer, predr)), '%')
print('TEXT ONLY ACCURACY SCORE IS: %0.2f' % (100 *  accuracy_score(true, pred)), '%')
print('ENSEMBLE ACCURACY SCORE (SAME WEIGHT) IS: %0.2f' % (100 *  accuracy_score(actual, predict)), '%')
print('ENSEMBLE ACCURACY SCORE WITH WEIGHT DISTRIBUTION', best_weight_ratio, '/', best_weight_text, 'IS: %0.2f' % (100 *  accuracy_score(actual, pred_best_weight)), '%')

RATIO ONLY ACCURACY SCORE IS: 72.00 %
TEXT ONLY ACCURACY SCORE IS: 80.00 %
ENSEMBLE ACCURACY SCORE (SAME WEIGHT) IS: 72.00 %
ENSEMBLE ACCURACY SCORE WITH WEIGHT DISTRIBUTION 0.3746 / 0.6254 IS: 80.00 %


In [24]:
# check overlapping predictions 

df_b = pd.DataFrame(bankrupted_risk_reports_test)
df_b.columns = ['file_name']
df_b['sec_code'] = df_b['file_name'].apply(lambda x:x.split('_')[0])
df_b['filing_date'] = df_b['file_name'].apply(lambda x:x.split('_')[3].split('.')[0])
df_b['filing_date'] = pd.to_datetime(df_b['filing_date'], format='%Y%m%d')
df_b = df_b.drop(['file_name', 'filing_date'], axis=1)
df_b['true class'] = 0

df_nb = pd.DataFrame(nonbankrupted_risk_reports_test)
df_nb.columns = ['file_name']
df_nb['sec_code'] = df_nb['file_name'].apply(lambda x:x.split('_')[0])
df_nb['filing_date'] = df_nb['file_name'].apply(lambda x:x.split('_')[3].split('.')[0])
df_nb['filing_date'] = pd.to_datetime(df_nb['filing_date'], format='%Y%m%d')
df_nb = df_nb.drop(['file_name', 'filing_date'], axis=1)
df_nb['true class'] = 1

df_both = [df_b, df_nb]
concatenated_D = pd.concat(df_both, ignore_index=True)
concatenated_D['ratio'] = pd.DataFrame(predr)
concatenated_D['text'] = pd.DataFrame(pred)
concatenated_D['ens_same'] = pd.DataFrame(predict)
concatenated_D['ens_weighted'] = pd.DataFrame(pred_best_weight)
print(concatenated_D)

   sec_code  true class  ratio  text  ens_same  ens_weighted
0      1744           0      0     0         0             0
1      1779           0      0     0         0             0
2      1872           0      0     0         0             0
3      1889           0      0     0         0             0
4      2758           0      1     0         1             0
5      4655           0      1     0         1             0
6      6667           0      0     0         0             0
7      8577           0      0     0         0             0
8      8866           0      0     0         0             0
9      8874           0      0     0         0             0
10     8900           0      0     0         0             0
11     8911           0      0     0         0             0
12     1780           1      1     1         1             1
13     1793           1      0     1         0             0
14     1911           1      0     1         1             1
15     2337           1 

In [25]:
# Results overlap into .csv

concatenated_D.to_csv('overlap_D.csv', index=False, encoding='utf-8')